In [1]:
from __future__ import print_function, division
import os
import torch
import torchvision
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch.nn as nn
import torch.nn.functional as F
from mpl_toolkits import mplot3d
import scipy.io


In [2]:
class fMRIDataset(Dataset):

    def _init_(self, mat_file, root_dir, transform = None):
        self.root_dir = root_dir
        self.fMRI_data = scipy.io.loadmat(root_dir+mat_file)['fmri_data_4d']
        self.vig_data = scipy.io.loadmat(root_dir+mat_file)['vigilance']
        self.transform = transform
 
    def __len__(self):
        """""""""
        Get the number of time points of one subject
        """""""""
        return len(self.fMRI_data[0,0,0,:])
    
    
    def __getitem__(self, idx):
        """
        Get the spatio data at idx time point
        
        """
        #print(self.fMRI_data.shape)
        data = self.fMRI_data[:,:,:,idx ]
        return data
    
    def getval(self,idx):
        """
        Get the target value of idx time point
        """
        val = self.vig_data[idx]
        return val
    
    def load_data(self, mat_file, root_dir, transform = None):
        """read mat_file, get 4d fmri data and vigilance data
            root_dir: the folder of subjects 
        """
        dataset = scipy.io.loadmat(root_dir+mat_file)
        self.fMRI_data = np.array(dataset['fmri_data_4d'])
        self.vig_data = np.array(dataset['vigilance'])
        
        #if len(self.fMRI_data) == 0 | len(self.vig_data) == 0:
        #   raise RuntimeError('Found 0 images, please check the data set')

        return (self.fMRI_data, self.vig_data)

In [3]:
## load training dataset 
## get train_ids 
#train_ids = next(os.walk(train_dir))[2]
#print(train_ids)
name =  'S7_Scan3_EC_fmri_and_vig'
train_dir = 'train/'
train_set = fMRIDataset()
dataset = train_set.load_data(name, train_dir, transform = None)
dataloader = DataLoader(train_set, batch_size = 4,  shuffle=False)

In [4]:
x = dataset[0]
y = dataset[1]

train_x = x[:,:,:,:300]
train_y = y[:300]

test_x = x[:,:,:,:300:]
test_y = y[300:]

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.model_zoo as model_zoo
import argparse
import os
from datetime import datetime
import socket
import timeit
from tensorboardX import SummaryWriter
import numpy as np
import torch
import torch.optim as optim

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [ ]:
class Net(nn.Module):
    def _init_(self):
        super(Net, self)._init_()
        
        self.conv1 = nn.Conv3d(16,33,3, stride = 2)
        #self.pool = nn.AvgPool2d(2,2,1)
        self.conv2 = nn.Conv3d(16,33,(3,5,2), stride = (2,1,1))
        
        #linear layer
        self.fc1 = nn.Linear(1280,100)
        self.fc2 = nn.Linear(100,10)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        
        x = x.view(-1, 1280)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        
        return x
    
net = Net()
net.to(device)

In [ ]:
loss_func = nn.CrossEntropyLoss()
opt = optim.SGD(net.parameters(), lr = 0.001, momentum= 0.9)

In [10]:
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error 
from matplotlib import pyplot as plt
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
from xgboost import XGBRegressor

In [24]:
NN_model = Sequential()
# The Input Layer :
NN_model.add(Dense(128, kernel_initializer='normal', input_shape = (61,73,300,61), activation='relu'))

# The Hidden Layers :
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))

# The Output Layer :
NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compile the network :
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
NN_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_26 (Dense)             (None, 61, 73, 300, 128)  7936      
_________________________________________________________________
dense_27 (Dense)             (None, 61, 73, 300, 256)  33024     
_________________________________________________________________
dense_28 (Dense)             (None, 61, 73, 300, 256)  65792     
_________________________________________________________________
dense_29 (Dense)             (None, 61, 73, 300, 256)  65792     
_________________________________________________________________
dense_30 (Dense)             (None, 61, 73, 300, 1)    257       
Total params: 172,801
Trainable params: 172,801
Non-trainable params: 0
_________________________________________________________________


In [21]:
checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

In [22]:
NN_model.fit(train_x, train_y, epochs=500, batch_size=32, validation_split = 0.2, callbacks=callbacks_list)

ValueError: Error when checking input: expected dense_16_input to have 5 dimensions, but got array with shape (61, 73, 61, 300)